# Traffic Sign Recognition

## Dependencies

In [1]:
import pandas as pd
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import models

#MANAGEMENT PURPOSES ONLY
from tqdm.notebook import tqdm

## Data management

In [2]:
data_path = r"./DATASET/"
img_size = 32

In [3]:
def load_data(dataset):
    images = []
    classes = []    
    rows = pd.read_csv(dataset)
    rows = rows.sample(frac=1).reset_index(drop=True)
            
    with tqdm(total=len(rows)+1) as pbar:
    
        for i, row in rows.iterrows():
            img_class = row["ClassId"]
            img_path = row["Path"]        
            image = os.path.join(data_path, img_path)

            image = cv2.imread(image)
            image_rs = cv2.resize(image, (img_size, img_size), 3)        

            R, G, B = cv2.split(image_rs)     

            img_r = cv2.equalizeHist(R)
            img_g = cv2.equalizeHist(G)
            img_b = cv2.equalizeHist(B)        

            new_image = cv2.merge((img_r, img_g, img_b))

            pbar.update(1)

            images.append(new_image)
            classes.append(img_class)
            
        pbar.container.children[-2].style.bar_color = '#00FF00' # Set bar to green color at end
    
    X = np.array(images)
    y = np.array(classes)
    
    return (X, y)

### Load data 

In [4]:
train_data_path = data_path + "Train.csv"
test_data_path = data_path + "Test.csv"


(Xtrain, Ytrain) = load_data(train_data_path)
(Xtest, Ytest) = load_data(test_data_path)

  0%|          | 0/39210 [00:00<?, ?it/s]

  0%|          | 0/12631 [00:00<?, ?it/s]

### Normalise data

In [5]:
Xtrain = Xtrain.astype("float32") / 255.0
Xtest = Xtest.astype("float32") / 255.0

num_labels = len(np.unique(Ytrain))
Ytrain = to_categorical(Ytrain, num_labels)
Ytest = to_categorical(Ytest, num_labels)

class_totals = Ytrain.sum(axis=0)
class_weight = class_totals.max() / class_totals

## Hyper parameters

In [19]:
epochs = 20
learning_rate = 0.001
batch_size = 64

#### Data augmentation

 Data augmentation creates modified versions of the images in our dataset. It allows us to add images to our dataset without us having to collect new ones

In [7]:
data_augmentation = ImageDataGenerator(
    rotation_range = 10,
    zoom_range = 0.15,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    shear_range = 0.15,
    horizontal_flip = False,
    vertical_flip = False
)

## Training Model

In [8]:
class RoadSignClassifier:
    
    def createCNN(width, height, depth, classes):
        
        """
         We will be using the Sequential API, 
         which allows us to create the model layer-by-layer
        """
        model = Sequential()
        inputShape = (height, width, depth)
        
        
        """
         First convolutional layer. Define output dim(8 X 8 X 3)
         Activation function “relu”
        """
        model.add(Conv2D(8, (5, 5), input_shape=inputShape, activation="relu"))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        
        """
        This time we include batch normalization. It just speeds up training.
        """
        model.add(Conv2D(16, (3, 3), activation="relu"))
        model.add(BatchNormalization())
        model.add(Conv2D(16, (3, 3), activation="relu"))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 2)))
 
        model.add(Conv2D(32, (3, 3), padding="same", activation="relu"))
        model.add(BatchNormalization())
        model.add(Conv2D(32, (3, 3), padding="same", activation="relu"))
        model.add(BatchNormalization())
        
        """
        The output in the final dense layer is equal to the number of classes that we have.
        """
        
        model.add(Flatten())
        model.add(Dropout(0.5))
        model.add(Dense(512, activation="relu"))        
        
        model.add(Dense(classes, activation="softmax"))
        
        return model

In [9]:
model = RoadSignClassifier.createCNN(
    width = img_size, 
    height = img_size, 
    depth = 3, 
    classes = len(class_totals)
)

optimizer = Adam(
    lr = learning_rate, 
    decay = (learning_rate / epochs)
)

In [10]:
model.compile(
    optimizer = optimizer, 
    loss = "categorical_crossentropy", 
    metrics = ["accuracy"]
)

In [11]:
fit = model.fit(
    
    data_augmentation.flow(
        Xtrain, 
        Ytrain, 
        batch_size = batch_size
    ), 
    epochs = epochs,
    
    validation_data = (Xtest, Ytest),
    
    class_weight = dict(enumerate(class_weight.flatten(), 0)),
    
    verbose=1
)

Epoch 1/20
613/613 [==============================] - 25s 41ms/step - loss: 4.3298 - accuracy: 0.4898 - val_loss: 0.9235 - val_accuracy: 0.7085
Epoch 2/20
613/613 [==============================] - 25s 41ms/step - loss: 1.8169 - accuracy: 0.7510 - val_loss: 0.4750 - val_accuracy: 0.8572
Epoch 3/20
613/613 [==============================] - 26s 42ms/step - loss: 1.2283 - accuracy: 0.8265 - val_loss: 0.3666 - val_accuracy: 0.8899
Epoch 4/20
613/613 [==============================] - 26s 42ms/step - loss: 0.8814 - accuracy: 0.8730 - val_loss: 0.5338 - val_accuracy: 0.8416
Epoch 5/20
613/613 [==============================] - 29s 47ms/step - loss: 0.7104 - accuracy: 0.8923 - val_loss: 0.3374 - val_accuracy: 0.9035
Epoch 6/20
613/613 [==============================] - 28s 46ms/step - loss: 0.6225 - accuracy: 0.9084 - val_loss: 0.3496 - val_accuracy: 0.9018
Epoch 7/20
613/613 [==============================] - 28s 45ms/step - loss: 0.5776 - accuracy: 0.9124 - val_loss: 0.3401 - val_accuracy:

In [20]:
model_name = "TSR_Model_96per"

In [21]:
model.save(f'./MODELS/{model_name}')

INFO:tensorflow:Assets written to: ./MODELS/TSR_Model_96per\assets


In [22]:
model = models.load_model(f'./MODELS/{model_name}')
model.compile(
    optimizer = optimizer, 
    loss = "categorical_crossentropy", 
    metrics = ["accuracy"]
)